# 模型组网

飞桨的模型组网分为通过内置模型组网，通过 Sequential 组网和通过 SubClass 组网三种形式，下面通过前面使用的LeNet网络分别介绍这三种形式。

## 通过内置模型组网

飞桨在 [paddle.vision.models.LeNet](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/vision/Overview_cn.html#about-models) 下内置了常用的分类模型，可以进行很方便的调用，通过下面的命令可以直接初始化一个LeNet模型

In [3]:
import paddle

lenet = paddle.vision.models.LeNet(num_classes=10)
print(lenet)

LeNet(
  (features): Sequential(
    (0): Conv2D(1, 6, kernel_size=[3, 3], padding=1, data_format=NCHW)
    (1): ReLU()
    (2): MaxPool2D(kernel_size=2, stride=2, padding=0)
    (3): Conv2D(6, 16, kernel_size=[5, 5], data_format=NCHW)
    (4): ReLU()
    (5): MaxPool2D(kernel_size=2, stride=2, padding=0)
  )
  (fc): Sequential(
    (0): Linear(in_features=400, out_features=120, dtype=float32)
    (1): Linear(in_features=120, out_features=84, dtype=float32)
    (2): Linear(in_features=84, out_features=10, dtype=float32)
  )
)


可以看到LeNet包含`features`和`fc`两个子网络，总共包含2个卷积层，2个ReLU激活层，2个MaxPool2D层，三个全链接层。

## 通过 Sequential 组网

针对顺序的线性网络结构,可以直接使用 [paddle.nn.Sequential](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Sequential_cn.html#sequential) 来快速完成组网，这种方式可以减少类的定义等代码编写。具体代码如下：

In [9]:
from paddle import nn

lenet_Sequential = nn.Sequential(
    nn.Conv2D(1, 6, 3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2D(2, 2),
    nn.Conv2D(6, 16, 5, stride=1, padding=0),
    nn.ReLU(),
    nn.MaxPool2D(2, 2),
    nn.Linear(400, 120),
    nn.Linear(120, 84), 
    nn.Linear(84, 10)
)
print(lenet_Sequential)

Sequential(
  (0): Conv2D(1, 6, kernel_size=[3, 3], padding=1, data_format=NCHW)
  (1): ReLU()
  (2): MaxPool2D(kernel_size=2, stride=2, padding=0)
  (3): Conv2D(6, 16, kernel_size=[5, 5], data_format=NCHW)
  (4): ReLU()
  (5): MaxPool2D(kernel_size=2, stride=2, padding=0)
  (6): Linear(in_features=400, out_features=120, dtype=float32)
  (7): Linear(in_features=120, out_features=84, dtype=float32)
  (8): Linear(in_features=84, out_features=10, dtype=float32)
)


## 通过 SubClass 组网

针对一些比较复杂的网络结构，就可以使用 SubClass 组网的方式来进行模型代码编写。通过 SubClass 组网进行组网需要完成下列三个步骤：
1. 创建一个继承自[paddle.nn.Layer](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Layer_cn.html#layer)
2. 在类的构造函数`__init__`中进行子Layer的定义，完成网络的构建
3. 在类的`forward`函数中使用定义的子Layer进行前向计算。

子Layer可以通过 基础API(卷积，池化或全连接)，Sequential 或 SubClass 的形式进行定义，子Layer在构造函数中一次定义后可在forward中多次调用。使用SubClass 组网形式实现LeNet的代码如下

In [11]:
class LeNet(nn.Layer):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2D(
                1, 6, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2D(2, 2),
            nn.Conv2D(
                6, 16, 5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2D(2, 2))

        if num_classes > 0:
            self.fc = nn.Sequential(
                nn.Linear(400, 120),
                nn.Linear(120, 84), nn.Linear(84, num_classes))

    def forward(self, inputs):
        x = self.features(inputs)

        if self.num_classes > 0:
            x = paddle.flatten(x, 1)
            x = self.fc(x)
        return x
lenet_SubClass = LeNet()
print(lenet_SubClass)

LeNet(
  (features): Sequential(
    (0): Conv2D(1, 6, kernel_size=[3, 3], padding=1, data_format=NCHW)
    (1): ReLU()
    (2): MaxPool2D(kernel_size=2, stride=2, padding=0)
    (3): Conv2D(6, 16, kernel_size=[5, 5], data_format=NCHW)
    (4): ReLU()
    (5): MaxPool2D(kernel_size=2, stride=2, padding=0)
  )
  (fc): Sequential(
    (0): Linear(in_features=400, out_features=120, dtype=float32)
    (1): Linear(in_features=120, out_features=84, dtype=float32)
    (2): Linear(in_features=84, out_features=10, dtype=float32)
  )
)


## 飞桨内置基础API

飞桨内置了大量基础的组网API，组网相关的API都在[paddle.nn](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Overview_cn.html#paddle-nn)目录下。组网相关的API类别与具体的API列表如下表：

| 功能          | API名称                                                                   |
| --- | ---|
| Conv          | Conv1D、Conv2D、Conv3D、Conv1DTranspose、Conv2DTranspose、Conv3DTranspose |
| Pool          | AdaptiveAvgPool1D、AdaptiveAvgPool2D、AdaptiveAvgPool3D、 AdaptiveMaxPool1D、AdaptiveMaxPool2D、AdaptiveMaxPool3D、 AvgPool1D、AvgPool2D、AvgPool3D、MaxPool1D、MaxPool2D、MaxPool3D          |
| Padding       | Pad1D、Pad2D、Pad3D                                                       |
| Activation    | ELU、GELU、Hardshrink、Hardtanh、HSigmoid、LeakyReLU、LogSigmoid、 LogSoftmax、PReLU、ReLU、ReLU6、SELU、Sigmoid、Softmax、Softplus、 Softshrink、Softsign、Tanh、Tanhshrink                                    |
| Normlization  | BatchNorm、BatchNorm1D、BatchNorm2D、BatchNorm3D、GroupNorm、 InstanceNorm1D、InstanceNorm2D、InstanceNorm3D、LayerNorm、SpectralNorm、 SyncBatchNorm                                                             |
| Recurrent NN  | BiRNN、GRU、GRUCell、LSTM、LSTMCell、RNN、RNNCellBase、SimpleRNN、SimpleRNNCell                    | 
| Transformer   | Transformer、TransformerDecoder、TransformerDecoderLayer、| TransformerEncoder、TransformerEncoderLayer |
| Dropout       | AlphaDropout、Dropout、Dropout2d、Dropout3d                               |
| Loss          | BCELoss、BCEWithLogitsLoss、CrossEntropyLoss、CTCLoss、KLDivLoss、L1Loss、 MarginRankingLoss、MSELoss、NLLLoss、SmoothL1Loss                         |

## 模型的参数

飞桨内置的 [paddle.summary](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/summary_cn.html#summary) 方法可以很方便的查看网络的基础结构，每层的输入输出shape和参数信息。

In [14]:
paddle.summary(lenet, (64, 1, 28, 28))

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-5      [[64, 1, 28, 28]]     [64, 6, 28, 28]          60       
    ReLU-5       [[64, 6, 28, 28]]     [64, 6, 28, 28]           0       
  MaxPool2D-5    [[64, 6, 28, 28]]     [64, 6, 14, 14]           0       
   Conv2D-6      [[64, 6, 14, 14]]     [64, 16, 10, 10]        2,416     
    ReLU-6       [[64, 16, 10, 10]]    [64, 16, 10, 10]          0       
  MaxPool2D-6    [[64, 16, 10, 10]]     [64, 16, 5, 5]           0       
   Linear-7         [[64, 400]]           [64, 120]           48,120     
   Linear-8         [[64, 120]]            [64, 84]           10,164     
   Linear-9          [[64, 84]]            [64, 10]             850      
Total params: 61,610
Trainable params: 61,610
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.19
Forward/backward

{'total_params': 61610, 'trainable_params': 61610}